# Multisig Wallet tutorial and playground with Python & web3.py

Tested on Energy Web Foundation's Tobalaba test network.

Please make 3 test accounts with some test tokens for experimentation

In [ ]:
import os
import time
import json
import datetime

from web3 import Web3
import web3.utils.events as eventutils

# address of wallet factory on the Tobalaba network
# needed if you do not want to deploy wallet contracts yourself
factoryWithDLAddress = Web3.toChecksumAddress("0x7d4ae47c29790f22f157982d84445fa8e2c6e178")

# folder with contract abis
contractfolder = os.path.join(os.getcwd(), "contracts")

# rpc address
http_rpc = "http://127.0.0.1:8545"

#reading abis
with open(os.path.join(contractfolder, 'MultiSigWalletWithDailyLimitFactory.json')) as f:
    factoryWithDLData = json.load(f)
    factoryWithDLAbi = factoryWithDLData["abi"]

with open(os.path.join(contractfolder, 'MultiSigWalletWithDailyLimit.json')) as f:
    walletWithDLData = json.load(f)
    walletWithDLAbi = walletWithDLData["abi"]

ADDRESS_EMPTY = '0x0000000000000000000000000000000000000000'

In [ ]:
# setting provider
w3 = Web3(Web3.HTTPProvider(http_rpc))

#set the default account to use and unlock it
w3.eth.defaultAccount = w3.eth.accounts[0]
#w3.personal.unlockAccount(w3.eth.defaultAccount, "yourpassword")

### 0. Let's create the factory

In [ ]:
factoryWithDailyLimit = w3.eth.contract(abi=factoryWithDLAbi, address=factoryWithDLAddress)

### 1. Let's create a multisig wallet
You need:
 - the owner account addresses
 - how many confirmations are needed to perform transactions
 - daily limit -> the amount that can be withrdrawn per day without the confirmation of others

These setting can be later changed

2 ways to create the wallet:
 1. using the deployed wallet factory (simpler)
 2. compiling and deploying the Wallet contract yourself
 
The first method is demonstrated here

##### Creation of the wallet

In [ ]:
address1 = Web3.toChecksumAddress(w3.eth.accounts[0])
address2 = Web3.toChecksumAddress(w3.eth.accounts[1])
address3 = Web3.toChecksumAddress(w3.eth.accounts[2])

requiredConfirmations = 2
dailyLimit = 0

txHash = factoryWithDailyLimit.functions.create([address1, address2, address3], requiredConfirmations, dailyLimit).transact()

The factory emits a ```ContractInstantiation(address sender, address instantiation)``` event in case of a newly created wallet. We can get the wallet's address by accessing it.
You can either read the event logs and parse the data from the transaction receipt, or set a filter for the event and scan for it. The first method is shown. Then the wallet contract is then instantiated using the address and its ABI.

In [ ]:
tx = w3.eth.getTransactionReceipt(txHash)
instantiationEvent = factoryWithDailyLimit.events.ContractInstantiation()
instEventLog = instantiationEvent.processReceipt(tx)

my_wallet_address = instEventLog[0].args["instantiation"]

my_wallet = w3.eth.contract(abi=walletWithDLAbi, address=my_wallet_address)

print("Your deployed wallet address is: " + my_wallet_address)
print("Owners are: " + str(my_wallet.functions.getOwners().call()))
print("Daily withdraw limit w/o confirmations: " + str(my_wallet.functions.dailyLimit().call()) + " wei")
print("Allowed withdraw for today w/o confirmations: " + str(my_wallet.functions.calcMaxWithdraw().call())+ " wei")
print("Required confirmations: " + str(my_wallet.functions.required().call()))

### 2. Let's send some play tokens to the wallet
E.g. 2 ethers for playing around

In [ ]:
w3.eth.sendTransaction({"from": w3.eth.accounts[0],
                        "to": my_wallet_address,
                        "value": w3.toWei(2, "ether")})

### 3. Let's try to withdraw some money back to account 1
 1. We submit a transaction invoking the ```submitTransaction(address destination, uint value, bytes data)```. ```value``` is where the transferrable 'money' goes in wei. More about the ```data``` field below in secton 5.
 2. We need to get the transaction ID: the wallet emits a ```Submission(uint indexed transactionId)``` event in case of  successful submission. We can read it out from the receipt.

In [ ]:
txHash = my_wallet.functions.submitTransaction(address1, w3.toWei(1, "ether"), bytes(0)).transact({"from": address1})
tx = w3.eth.getTransactionReceipt(txHash)
submissionEvent = my_wallet.events.Submission()
subEventLog = submissionEvent.processReceipt(tx)
transactionId = subEventLog[0].args['transactionId']

print("Transaction id: "+ str(transactionId))

#### 3.1 Let's check out the state of our submission

In [ ]:
print("Transaction count: " + str(my_wallet.functions.transactionCount().call()))
print("Our transaction [destination/value/data/executed]: " + str(my_wallet.functions.transactions(transactionId).call()))
print("Confirmed by {}: {}".format(address1, my_wallet.functions.confirmations(transactionId, address1).call()))
print("Confirmed by {}: {}".format(address2, my_wallet.functions.confirmations(transactionId, address2).call()))
print("Confirmed by {}: {}".format(address3, my_wallet.functions.confirmations(transactionId, address3).call()))

#### 3.2 You can check that as long as the submission is not confirmed by at least another owner, you cannot send it

In [ ]:
my_wallet.functions.executeTransaction(transactionId).transact({"from": address1, "gas": 800000})
time.sleep(5)
print("Our transaction [destination/value/data/executed]: " + str(my_wallet.functions.transactions(transactionId).call()))

### 4. Confirm the transaction with other owners
 - ```confirmTransaction(uint transactionId)```
 - needs the transaction ID, and the sender needs to be the owner who confirms
 - a confirmed transaction can be executed by ``` executeTransaction(transactionId)```
 - ``` executeTransaction(transactionId)``` is automatically triggered if the number of confirmations reach the required with this last ```confirmTransaction``` and all conditions are met


In [ ]:
my_wallet.functions.confirmTransaction(transactionId).transact({"from": address2, "gas": 800000})

In [ ]:
print("Our transaction [destination/value/data/executed]: " + str(my_wallet.functions.transactions(transactionId).call()))

You can see that the transaction status is executed and the money appears on you destination account.

### 5. Change daily limit 
You cannot calll the wallet's changeDailyLimit function directly, it needs to be a confirmed transaction. You can change the daily limit, owners and required confirmation as well, with the consent of the owners.

#### How to invoke Smart Contract methods using your Multisig Wallet

If you look at the function signature ```submitTransaction(address destination, uint value, bytes data)``` you notice a ```bytes data``` field. It is used to invoke functionalities of a contract and can be left empty or bytes(0) for regular value transfers. Remember that invoking SC functions is just a regular transaction containing the relevant calldata. Calldata is obtained by encoding the desired function's signature and its parameters, but fortunately web3 libraries already do the heavy lifting for you, so no need to do this manually. The recipient address needs to be the address of the Smart Contract whose method you want to invoke.

In this case we want to invoke the ```changeDailyLimit``` function of our wallet Smart Contract. We need the ABI to encode the calldata easily.

In [ ]:
call_data = my_wallet.encodeABI(fn_name='changeDailyLimit', args=[w3.toWei(1, "ether")])
txHash=my_wallet.functions.submitTransaction(my_wallet_address, 0, w3.toBytes(hexstr=call_data)).transact({"from": address1})

After this points everything goes as with any other transaction from our wallet

In [ ]:
tx = w3.eth.getTransactionReceipt(txHash)
submissionEvent = my_wallet.events.Submission()
subEventLog = submissionEvent.processReceipt(tx)
transactionId = subEventLog[0].args['transactionId']

print("Transaction id: "+ str(transactionId))

We confirm it with another account as well, which triggers the transaction

In [ ]:
my_wallet.functions.confirmTransaction(transactionId).transact({"from": address2, "gas": 800000})
time.sleep(7)
if(my_wallet.functions.transactions(transactionId).call()[3] == True):
    print("transaction {} is executed".format(transactionId))
    print("Daily withdraw limit w/o confirmations: " + str(my_wallet.functions.dailyLimit().call()) + " wei")
else:
    print("transaction {} is not executed".format(transactionId))

You should see that the daily limit has changed

### 6. Withdraw some ether w/o confirmation
##### Now that the daily limit is changed, it is time to test it. Calldata is 0.

In [ ]:
txHash = my_wallet.functions.submitTransaction(address1, w3.toWei(0.7, "ether"), bytes(0)).transact({"from": address1})

tx = w3.eth.getTransactionReceipt(txHash)
submissionEvent = my_wallet.events.Submission()
subEventLog = submissionEvent.processReceipt(tx)
transactionId = subEventLog[0].args['transactionId']

print("Transaction id: "+ str(transactionId))

In [ ]:
print("Transaction count: " + str(my_wallet.functions.transactionCount().call()))
print("Our transaction [destination/value/data/executed]: " + str(my_wallet.functions.transactions(transactionId).call()))
print("Confirmed by {}: {}".format(address1, my_wallet.functions.confirmations(transactionId, address1).call()))
print("Confirmed by {}: {}".format(address2, my_wallet.functions.confirmations(transactionId, address2).call()))
print("Confirmed by {}: {}".format(address3, my_wallet.functions.confirmations(transactionId, address3).call()))

##### You can see that the transaction is executed, but only 1 account confirmed it. Let's check the remaining daily quota.
The daily quota is calculated for "today" by comparing the current time to a unix timestamp called ```lastDay```. If the curent moment is past ```lastDay + 24 hours``` then the daily quota resets. The lastDay timestamp is initially zero and is first set in the contract when we try to make a withdrawal.

In [ ]:
print("Daily withdraw limit w/o confirmations: " + str(my_wallet.functions.dailyLimit().call()) + " wei")
print("Allowed withdraw for today w/o confirmations: " + str(my_wallet.functions.calcMaxWithdraw().call())+ " wei")
last_day = my_wallet.functions.lastDay().call()
print("Last day: {} - {}".format(last_day, datetime.datetime.fromtimestamp(int(last_day))))

### 7. Let's remove an owner
Steps are very similar to changing the daily limit. You can manage ownership with```removeOwner(address owner)``` and```replaceOwner(address owner, address newOwner)``` methods, and change the confirmations needed with ```changeRequirement(uint _required)```.

I remove owner n3 in this example

In [ ]:
call_data = my_wallet.encodeABI(fn_name='removeOwner', args=[address3])
txHash=my_wallet.functions.submitTransaction(my_wallet_address, 0, w3.toBytes(hexstr=call_data)).transact({"from": address1})

In [ ]:
tx = w3.eth.getTransactionReceipt(txHash)
submissionEvent = my_wallet.events.Submission()
subEventLog = submissionEvent.processReceipt(tx)
transactionId = subEventLog[0].args['transactionId']

print("Transaction id: "+ str(transactionId))

In [ ]:
my_wallet.functions.confirmTransaction(transactionId).transact({"from": address2, "gas": 800000})
time.sleep(7)
if(my_wallet.functions.transactions(transactionId).call()[3] == True):
    print("transaction {} is executed".format(transactionId))
    print("Owners are: " + str(my_wallet.functions.getOwners().call()))
else:
    print("transaction {} is not executed".format(transactionId))

##### You should see the chosen owner disappeared from the list

### 8. Your experiments here
Feel free to play around with your Multisig wallet